In [1]:
# nuclio: ignore
import nuclio

In [3]:
from mlrun import new_function, code_to_function, get_run_db, mount_v3io, mlconf, new_model_server, v3io_cred
import os
# for local DB path use '/User/mlrun' instead 
mlconf.dbpath = '/User/mlrun-db'

In [18]:
import datetime

## Define functions for pipeline

In [12]:
generator_fn = code_to_function(name='generator',
                                runtime='nuclio:mlrun',
                                filename='nuclio-generator.ipynb')
generator_fn.add_volume('User','~/')

In [13]:
preprocessing_fn = code_to_function(name='preprocessing',
                                    runtime='nuclio:mlrun',
                                    filename='nuclio-data-preperations.ipynb')
preprocessing_fn.add_volume('User','~/')

In [14]:
training_fn = code_to_function(name='training',
                                    runtime='nuclio:mlrun',
                                    filename='nuclio-training.ipynb')
training_fn.add_volume('User','~/')

In [15]:
inference_fn = code_to_function(name='inference',
                                runtime='nuclio:mlrun',
                                filename='nuclio-inference.ipynb')
inference_fn.add_volume('User','~/')

## Create and run the pipeline

In [10]:
import kfp
from kfp import dsl

In [11]:
artifacts_path = 'v3io:///users/admin/mlrun-db/kfp/{{workflow.uid}}/'

In [19]:
@dsl.pipeline(
    name='Network Operations Training Pipeline',
    description='Network operations E2E pipeline'
)
def netops_pipepline(
    use_tsdb = 1,
    metrics_configuration_file = os.path.join(os.getcwd(), 
                                              'configurations', 
                                              'metrics_configuration.yaml'),
    deployment_table = 'netops-deployment',
    metrics_table = 'netops-metrics',
    initial_timestamp = str(int((datetime.datetime.now()-datetime.timedelta(days=1)).timestamp())),
    SECS_TO_GENERATE = 60*60*60
):
    
    generator = generator_fn.as_step(name='generator',
                                     params={'use_tsdb': use_tsdb,
                                             'metrics_configuration_file': metrics_configuration_file,
                                             'deployment_table': deployment_table,
                                             'metrics_table': metrics_table,
                                             'initial_timestamp': initial_timestamp,
                                             'SECS_TO_GENERATE': SECS_TO_GENERATE})

In [21]:
client = kfp.Client(namespace='default-tenant')
arguments = {}
run_result = client.create_run_from_pipeline_func(netops_pipepline, 
                                                  arguments, 
                                                  experiment_name='netops-mlrun')

/User/.pythonlibs/lib/python3.6/site-packages/kfp/components/_data_passing.py:133: UserWarning: Missing type name was inferred as "Integer" based on the value "1".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/User/.pythonlibs/lib/python3.6/site-packages/kfp/components/_data_passing.py:133: UserWarning: Missing type name was inferred as "Integer" based on the value "216000".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
